## Flatiron Capstone Nom-Native

SyntaxError: invalid syntax (2773187961.py, line 1)

In [89]:
### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
sns.color_palette("Blues", as_cmap=True)

### Standard Packages
import numpy as np
import warnings
import nltk
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

### NLTK
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet')
nltk.download('vader_lexicon')

### Scikit-Learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, \
                            accuracy_score, f1_score, recall_score, precision_score

### Streamlit
import streamlit as st

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jake\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jake\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Jake\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
### Creating a function that makes all text lowercase for further analysis

def lower_case(text):
    text = text.lower()
    return text

### Creating a function that removes the use of via in the context of via hashtag or via mention (removes 80% of vias)

def remove_via(text):
    
    if 'via @' in text or 'via #' in text:
        text = text.replace('via', '')
    return text
    
### Creating a function that removes errant html syntax from the tweet (e.g. &amp; and &quot;)

def remove_html(text):
    
    text = text.replace('&amp;', '')
    text = text.replace('&quot;', '')
    return text

### Creating a function that removes urls or instances of '{link}' from the tweet

def remove_url(text):
    
    url_like_strings = ['{link}', '.com', 'http', 'bit.ly', '.co']
    text = text.split()
    for s in url_like_strings:
        text = [word for word in text if s not in word]
    text = ' '.join(text)
    return text

### Creating a function that removes words that contain a @ and rt (retweet) 
### as mentions would not be important in determining the emotion of a tweet

def remove_at_and_rt(text):
    text = text.split()
    text = [word for word in text if '@' not in word]
    text = [word for word in text if word != 'rt']
    text = ' '.join(text)
    return text

### Creating a function that removes '#SXSW' of any case type from the text

def remove_sxsw(text):
    text = text.split()
    text = [word for word in text if '#sxsw' not in word]
    text = ' '.join(text)
    return text

### Creating a function that uses a regex tokenizer to remove punctuation but ignores contraction apostrophes

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+\'?\w+')
    text = tokenizer.tokenize(text)
    text = ' '.join(text)
    return text

### Creating a function that removes stopwords from a specified list of stopwords

custom_stop_words = ['in','of','at','a','the']

def remove_stopwords(text, stop_words_list = set(stopwords.words('english'))):
    text = text.split()
    text = [word for word in text if word not in stop_words_list]
    text = ' '.join(text)
    return text

### Creating a function that removes non-ASCII characters

def remove_characters(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

### Creating a function that lemmatizes words

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

### Creating a function that combines all of the above functions

def clean_text(text):
    text = lower_case(text)
    text = remove_via(text)
    text = remove_html(text)
    text = remove_url(text)
    text = remove_at_and_rt(text)
    text = remove_sxsw(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_characters(text)
    text = lemmatize(text)
    return text

In [5]:
df = pd.read_csv('Data/yelp_review.csv')
df = pd.DataFrame(df)

In [6]:
df = pd.DataFrame(df)

In [7]:
df

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly.,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu every so often. Cool decor and vibe inside their 30 seat restaurant. Call for a reservation. \n\nWe had their beef tartar and pork belly to start and a salmon dish and lamb meal for mains. Everything was incredible! I could go on at length about how all the listed ingredients really make their dishes amazing but honestly you just need to go. \n\nA bit outside of downtown montreal but take the metro out and it's less than a 10 minute walk from the station.,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhood and has been there since 1951. They are known for smoked meat which most deli's have but their brisket sandwich is what I come to montreal for. They've got about 12 seats outside to go along with the inside. \n\nThe smoked meat is up there in quality and taste with Schwartz's and you'll find less tourists at Lester's as well.,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,"Love coming here. Yes the place always needs the floor swept but when you give out peanuts in the shell how won't it always be a bit dirty. \n\nThe food speaks for itself, so good. Burgers are made to order and the meat is put on the grill when you order your sandwich. Getting the small burger just means 1 patty, the regular is a 2 patty burger which is twice the deliciousness. \n\nGetting the Cajun fries adds a bit of spice to them and whatever size you order they always throw more fries (a lot more fries) into the bag.",0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.\n\nIf you're looking for a light breakfast then head out here. Perfect spot for a coffee\/latté before heading out to the old port,0,0,0
...,...,...,...,...,...,...,...,...,...
5261663,PoGSiNz1X5SUu0qEt-qM5w,mPjPyipaD0C_myqWqDipZg,Ngk84Ax1tXgpoJFEGxot3w,1,2011-11-21,"Bought groupon $39 for 4 months from groupon, the sign up guy tried so hard to sell a 'new' package to me. I asked him if I can make up my mind after I tried the gym out first, but he insisted I do it now or else the price will go up. As a matter of fact, he has a price increase meeting tomorrow....hahaha\n\nDid not opt for the 'new' package, stuck with my original package. He seemed upset...said fine, your membership expires in 4 months, you just get your membership and nothing else, you will have to pay for parking too. I told him...but I see that there is a sign that says one hr free parking. umm...err...ehhh...yeah, well, only one hr free, after that you will have to pay $3\/hr. whatever...\n\nAfter signing everything, he said you will get your access card after you finish your assessment (aka personal trainer sales pitch). I told him I do not need an assessment as I will only be using the treadmill. He said I HAVE to get an assessment for insurance purpose....LAAAME!\n\nwill update after my physical assessment.",36,3,3
5261664,-CJNPrDWgIkorx4iEZJXIg,mPjPyipaD0C_myqWqDipZg,pOEL97ld-FJMKO8Ki8JmYg,3,2016-04-30,"Spring rolls was pretty good, cod was a bit raw\/cold in the middle. Server was too cool..... As if he's doing us a favour.",0,0,0
5261665,W9eVvOcpBvG6lpJPoJOxuA,mPjPyipaD0C_myqWqDipZg,5ubokMNw8qfbX2WtxgJG1Q,4,2011-10-23,"Had a 8 dish set meal, was enough for 10 people for $126. \n\n-Seafood Soup\n-Sweet and Sour pork\n-Scallop with greens\n-Steamed fish\n-Lo

In [8]:
df['business_id'].unique()

array(['AEx2SYEUJmTxVVB18LlCwA', 'VR6GpWIda3SfvPC-lg9H3w',
       'CKC0-MOWMqoeWf6s-szl8g', ..., 'ALJGJTsi2hxEH4f3_Fxo4g',
       'GQMnv10QTLeCpCHkLBcVUg', 'UPJM8thQlVAb1OASCCRfLA'], dtype=object)

In [9]:
df = df.drop(['review_id', 'user_id', 'date'], axis=1)

In [10]:
#sid = SentimentIntensityAnalyzer()
#df['sentiment'] = df['text'].apply(lambda x: sid.polarity_scores(x))
#df = pd.concat([df.drop(['sentiment'], axis=1), df['sentiment'].apply(pd.Series)], axis=1)

In [11]:
df

,business_id,stars,text,useful,funny,cool
0,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly.,0,0,0
1,VR6GpWIda3SfvPC-lg9H3w,5,Small unassuming place that changes their menu every so often. Cool decor and vibe inside their 30 seat restaurant. Call for a reservation. \n\nWe had their beef tartar and pork belly to start and a salmon dish and lamb meal for mains. Everything was incredible! I could go on at length about how all the listed ingredients really make their dishes amazing but honestly you just need to go. \n\nA bit outside of downtown montreal but take the metro out and it's less than a 10 minute walk from the station.,0,0,0
2,CKC0-MOWMqoeWf6s-szl8g,5,Lester's is located in a beautiful neighborhood and has been there since 1951. They are known for smoked meat which most deli's have but their brisket sandwich is what I come to montreal for. They've got about 12 seats outside to go along with the inside. \n\nThe smoked meat is up there in quality and taste with Schwartz's and you'll find less tourists at Lester's as well.,0,0,0
3,ACFtxLv8pGrrxMm6EgjreA,4,"Love coming here. Yes the place always needs the floor swept but when you give out peanuts in the shell how won't it always be a bit dirty. \n\nThe food speaks for itself, so good. Burgers are made to order and the meat is put on the grill when you order your sandwich. Getting the small burger just means 1 patty, the regular is a 2 patty burger which is twice the deliciousness. \n\nGetting the Cajun fries adds a bit of spice to them and whatever size you order they always throw more fries (a lot more fries) into the bag.",0,0,0
4,s2I_Ni76bjJNK9yG60iD-Q,4,Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.\n\nIf you're looking for a light breakfast then head out here. Perfect spot for a coffee\/latté before heading out to the old port,0,0,0
...,...,...,...,...,...,...
5261663,Ngk84Ax1tXgpoJFEGxot3w,1,"Bought groupon $39 for 4 months from groupon, the sign up guy tried so hard to sell a 'new' package to me. I asked him if I can make up my mind after I tried the gym out first, but he insisted I do it now or else the price will go up. As a matter of fact, he has a price increase meeting tomorrow....hahaha\n\nDid not opt for the 'new' package, stuck with my original package. He seemed upset...said fine, your membership expires in 4 months, you just get your membership and nothing else, you will have to pay for parking too. I told him...but I see that there is a sign that says one hr free parking. umm...err...ehhh...yeah, well, only one hr free, after that you will have to pay $3\/hr. whatever...\n\nAfter signing everything, he said you will get your access card after you finish your assessment (aka personal trainer sales pitch). I told him I do not need an assessment as I will only be using the treadmill. He said I HAVE to get an assessment for insurance purpose....LAAAME!\n\nwill update after my physical assessment.",36,3,3
5261664,pOEL97ld-FJMKO8Ki8JmYg,3,"Spring rolls was pretty good, cod was a bit raw\/cold in the middle. Server was too cool..... As if he's doing us a favour.",0,0,0
5261665,5ubokMNw8qfbX2WtxgJG1Q,4,"Had a 8 dish set meal, was enough for 10 people for $126. \n\n-Seafood Soup\n-Sweet and Sour pork\n-Scallop with greens\n-Steamed fish\n-Lobster - ehh...oooh kay\n-Some rotisserie chicken with some sauce - Tastes pretty good\n-Duck covered with taro and deep fried - nothing to write home about, more taro than duck\n\nComplimentary dessert - red bean and small tapioca (EXCELLENT)\n\nEverything was pretty fresh and most importantly hot (temperature) when it came out.\n\nTaste wise, it's pretty much taste like any other cantonese cuisine.\n\nService was pretty fast and efficient, and both of the waiters we dealt spoke english

In [12]:
#df.to_csv('data/reviewvader.csv')

In [13]:
df1 = pd.read_csv('Data/reviewvader.csv')

In [14]:
#df1 = df1[~df1['stars'].isin([1, 2])]

In [15]:
df1

,Unnamed: 0,business_id,stars,text,useful,funny,cool,neg,neu,pos,compound
0,0,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly.,0,0,0,0.000,0.645,0.355,0.9551
1,1,VR6GpWIda3SfvPC-lg9H3w,5,Small unassuming place that changes their menu every so often. Cool decor and vibe inside their 30 seat restaurant. Call for a reservation. \n\nWe had their beef tartar and pork belly to start and a salmon dish and lamb meal for mains. Everything was incredible! I could go on at length about how all the listed ingredients really make their dishes amazing but honestly you just need to go. \n\nA bit outside of downtown montreal but take the metro out and it's less than a 10 minute walk from the station.,0,0,0,0.000,0.907,0.093,0.8167
2,2,CKC0-MOWMqoeWf6s-szl8g,5,Lester's is located in a beautiful neighborhood and has been there since 1951. They are known for smoked meat which most deli's have but their brisket sandwich is what I come to montreal for. They've got about 12 seats outside to go along with the inside. \n\nThe smoked meat is up there in quality and taste with Schwartz's and you'll find less tourists at Lester's as well.,0,0,0,0.000,0.925,0.075,0.6249
3,3,ACFtxLv8pGrrxMm6EgjreA,4,"Love coming here. Yes the place always needs the floor swept but when you give out peanuts in the shell how won't it always be a bit dirty. \n\nThe food speaks for itself, so good. Burgers are made to order and the meat is put on the grill when you order your sandwich. Getting the small burger just means 1 patty, the regular is a 2 patty burger which is twice the deliciousness. \n\nGetting the Cajun fries adds a bit of spice to them and whatever size you order they always throw more fries (a lot more fries) into the bag.",0,0,0,0.036,0.842,0.123,0.8560
4,4,s2I_Ni76bjJNK9yG60iD-Q,4,Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.\n\nIf you're looking for a light breakfast then head out here. Perfect spot for a coffee\/latté before heading out to the old port,0,0,0,0.000,0.826,0.174,0.8313
...,...,...,...,...,...,...,...,...,...,...,...
5261663,5261663,Ngk84Ax1tXgpoJFEGxot3w,1,"Bought groupon $39 for 4 months from groupon, the sign up guy tried so hard to sell a 'new' package to me. I asked him if I can make up my mind after I tried the gym out first, but he insisted I do it now or else the price will go up. As a matter of fact, he has a price increase meeting tomorrow....hahaha\n\nDid not opt for the 'new' package, stuck with my original package. He seemed upset...said fine, your membership expires in 4 months, you just get your membership and nothing else, you will have to pay for parking too. I told him...but I see that there is a sign that says one hr free parking. umm...err...ehhh...yeah, well, only one hr free, after that you will have to pay $3\/hr. whatever...\n\nAfter signing everything, he said you will get your access card after you finish your assessment (aka personal trainer sales pitch). I told him I do not need an assessment as I will only be using the treadmill. He said I HAVE to get an assessment for insurance purpose....LAAAME!\n\nwill update after my physical assessment.",36,3,3,0.037,0.852,0.111,0.9429
5261664,5261664,pOEL97ld-FJMKO8Ki8JmYg,3,"Spring rolls was pretty good, cod was a bit raw\/cold in the middle. Server was too cool..... As if he's doing us a favour.",0,0,0,0.000,0.679,0.321,0.8402
5261665,5261665,5ubokMNw8qfbX2WtxgJG1Q,4,"Had a 8 dish set meal, was enough for 10 people for $126. \n\n-Seafood Soup\n-Sweet and Sour pork\n-Scallop with greens\n-Steamed fish\n-Lobster - ehh...oooh kay\n-Some rotisserie chicken with some sauce - Tastes pretty good\n-Duck covered with taro and deep fried - nothing to write home about, more taro than duck\n\nComplimentary dessert - red bean and small tapioca

In [16]:
print('BEFORE \n' + df1.text[0] + '\n')
print('AFTER \n' + clean_text(df1['text'][0]))


BEFORE 
Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. 

Staff was very helpful and friendly.

AFTER 
super simple place amazing nonetheless around since 30's still serve thing started bologna salami sandwich mustard staff helpful friendly


In [17]:
df1

,Unnamed: 0,business_id,stars,text,useful,funny,cool,neg,neu,pos,compound
0,0,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly.,0,0,0,0.000,0.645,0.355,0.9551
1,1,VR6GpWIda3SfvPC-lg9H3w,5,Small unassuming place that changes their menu every so often. Cool decor and vibe inside their 30 seat restaurant. Call for a reservation. \n\nWe had their beef tartar and pork belly to start and a salmon dish and lamb meal for mains. Everything was incredible! I could go on at length about how all the listed ingredients really make their dishes amazing but honestly you just need to go. \n\nA bit outside of downtown montreal but take the metro out and it's less than a 10 minute walk from the station.,0,0,0,0.000,0.907,0.093,0.8167
2,2,CKC0-MOWMqoeWf6s-szl8g,5,Lester's is located in a beautiful neighborhood and has been there since 1951. They are known for smoked meat which most deli's have but their brisket sandwich is what I come to montreal for. They've got about 12 seats outside to go along with the inside. \n\nThe smoked meat is up there in quality and taste with Schwartz's and you'll find less tourists at Lester's as well.,0,0,0,0.000,0.925,0.075,0.6249
3,3,ACFtxLv8pGrrxMm6EgjreA,4,"Love coming here. Yes the place always needs the floor swept but when you give out peanuts in the shell how won't it always be a bit dirty. \n\nThe food speaks for itself, so good. Burgers are made to order and the meat is put on the grill when you order your sandwich. Getting the small burger just means 1 patty, the regular is a 2 patty burger which is twice the deliciousness. \n\nGetting the Cajun fries adds a bit of spice to them and whatever size you order they always throw more fries (a lot more fries) into the bag.",0,0,0,0.036,0.842,0.123,0.8560
4,4,s2I_Ni76bjJNK9yG60iD-Q,4,Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.\n\nIf you're looking for a light breakfast then head out here. Perfect spot for a coffee\/latté before heading out to the old port,0,0,0,0.000,0.826,0.174,0.8313
...,...,...,...,...,...,...,...,...,...,...,...
5261663,5261663,Ngk84Ax1tXgpoJFEGxot3w,1,"Bought groupon $39 for 4 months from groupon, the sign up guy tried so hard to sell a 'new' package to me. I asked him if I can make up my mind after I tried the gym out first, but he insisted I do it now or else the price will go up. As a matter of fact, he has a price increase meeting tomorrow....hahaha\n\nDid not opt for the 'new' package, stuck with my original package. He seemed upset...said fine, your membership expires in 4 months, you just get your membership and nothing else, you will have to pay for parking too. I told him...but I see that there is a sign that says one hr free parking. umm...err...ehhh...yeah, well, only one hr free, after that you will have to pay $3\/hr. whatever...\n\nAfter signing everything, he said you will get your access card after you finish your assessment (aka personal trainer sales pitch). I told him I do not need an assessment as I will only be using the treadmill. He said I HAVE to get an assessment for insurance purpose....LAAAME!\n\nwill update after my physical assessment.",36,3,3,0.037,0.852,0.111,0.9429
5261664,5261664,pOEL97ld-FJMKO8Ki8JmYg,3,"Spring rolls was pretty good, cod was a bit raw\/cold in the middle. Server was too cool..... As if he's doing us a favour.",0,0,0,0.000,0.679,0.321,0.8402
5261665,5261665,5ubokMNw8qfbX2WtxgJG1Q,4,"Had a 8 dish set meal, was enough for 10 people for $126. \n\n-Seafood Soup\n-Sweet and Sour pork\n-Scallop with greens\n-Steamed fish\n-Lobster - ehh...oooh kay\n-Some rotisserie chicken with some sauce - Tastes pretty good\n-Duck covered with taro and deep fried - nothing to write home about, more taro than duck\n\nComplimentary dessert - red bean and small tapioca

In [18]:
result = df1.groupby('business_id')['neg', 'neu', 'pos', 'stars'].mean().reset_index()

print(result)

                   business_id       neg       neu       pos     stars
0       --6MefnULPED_I942VcFNA  0.039351  0.805459  0.155108  3.135135
1       --7zmmkVg-IMGaXbuVd0SQ  0.039043  0.718319  0.242809  3.851064
2       --8LPVSo5i0Oo61X01sV9A  0.058000  0.711333  0.230333  4.333333
3       --9QQLMTbFzLJ_oT-ON3Xw  0.065455  0.743273  0.191364  3.363636
4       --9e1ONYQuAa-CB_Rrw7Tw  0.036409  0.747744  0.215857  4.088904
...                        ...       ...       ...       ...       ...
174562  zzvlwkcNR1CCqOPXwuvz2A  0.048667  0.795333  0.156333  3.000000
174563  zzwaS0xn1MVEPEf0hNLjew  0.057266  0.743547  0.199203  3.437500
174564  zzwhN7x37nyjP0ZM8oiHmw  0.028784  0.692324  0.278865  4.108108
174565  zzwicjPC9g246MK2M1ZFBA  0.061118  0.746255  0.192588  2.921569
174566  zzzaIBwimxVej4tY6qFOUQ  0.029811  0.748270  0.221946  3.432432

[174567 rows x 5 columns]


In [19]:
result.to_csv('result.csv', index=False)

In [20]:
df2 = pd.read_csv('result.csv')
df2 = pd.DataFrame(df2)

In [21]:
df2

,business_id,neg,neu,pos,stars
0,--6MefnULPED_I942VcFNA,0.039351,0.805459,0.155108,3.135135
1,--7zmmkVg-IMGaXbuVd0SQ,0.039043,0.718319,0.242809,3.851064
2,--8LPVSo5i0Oo61X01sV9A,0.058000,0.711333,0.230333,4.333333
3,--9QQLMTbFzLJ_oT-ON3Xw,0.065455,0.743273,0.191364,3.363636
4,--9e1ONYQuAa-CB_Rrw7Tw,0.036409,0.747744,0.215857,4.088904
...,...,...,...,...,...
174562,zzvlwkcNR1CCqOPXwuvz2A,0.048667,0.795333,0.156333,3.000000
174563,zzwaS0xn1MVEPEf0hNLjew,0.057266,0.743547,0.199203,3.437500
174564,zzwhN7x37nyjP0ZM8oiHmw,0.028784,0.692324,0.278865,4.108108
174565,zzwicjPC9g246MK2M1ZFBA,0.061118,0.746255,0.192588,2.921569


In [22]:
df3 = pd.read_csv('merged.csv')

In [23]:
merged_df = pd.merge(df2, df3, on='business_id')


In [24]:
allowed_values = ['NC', 'OH']

In [25]:
df_filtered = merged_df[merged_df['state'].isin(allowed_values)]

In [26]:
df_filtered

,business_id,neg,neu,pos,stars_x,name,neighborhood,address,city,state,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
1,--7zmmkVg-IMGaXbuVd0SQ,0.039043,0.718319,0.242809,3.851064,"""Primal Brewery""",NaN,"""16432 Old Statesville Rd""",Huntersville,NC,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
7,--EF5N7P70J_UYBTPypYlA,0.081000,0.788000,0.131000,3.000000,"""MV Nail Spa""",NaN,"""24139 Lorain Rd""",North Olmsted,OH,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
8,--EX4rRznJrltyn-34Jz1w,0.014250,0.744750,0.241000,4.000000,"""Bath & Body Works""",NaN,"""6801 Northlake Mall Dr, Ste 172""",Charlotte,NC,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
12,--I7YYLada0tSLkORTHb5Q,0.040431,0.743097,0.216458,3.486111,"""Happy Moose Bar and Grill""",NaN,"""9436 State Rte 14""",Streetsboro,OH,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
13,--KCl2FvVQpvjzmZSPyviA,0.045917,0.752750,0.201333,2.916667,"""Hungry Howie's Pizza""",NaN,"""6426 W Sugar Creek Rd, Unit A""",Charlotte,NC,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152014,zzSbDY2Ztja3Htt8bsQ6Aw,0.069500,0.786417,0.144167,3.416667,"""Dairy King""",NaN,"""6484 Brecksville Rd""",Independence,OH,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152015,zzTM2KyJkKomLDkl6rM4dQ,0.053625,0.835250,0.111125,3.000000,"""Mike The Plumber""",NaN,"""6126 Deveron Dr""",Charlotte,NC,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152022,zzbsLej_8KtTxGwJdOb09w,0.168000,0.796000,0.036000,1.666667,"""Wendy's""",NaN,"""5855 Som Ctr Rd""",Willoughby,OH,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152030,zzm9aR0UHEEEYdsR_zKcuw,0.011000,0.679500,0.309500,5.000000,"""Serenity Hair Spa by Lauren""",Ballantyne,"""12335 N Community House Rd, Ste 14""",Charlotte,NC,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [27]:
columns_to_drop = ['neighborhood', 'is_open', 'address', 'business_id', 'city', 'postal_code', 'latitude', 'longitude', ]

In [28]:
df_filtered = df_filtered.drop(columns_to_drop, axis=1)
df_filtered

,neg,neu,pos,stars_x,name,state,stars_y,review_count,categories,AcceptsInsurance,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
1,0.039043,0.718319,0.242809,3.851064,"""Primal Brewery""",NC,4.0,47,Food;Breweries,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
7,0.081000,0.788000,0.131000,3.000000,"""MV Nail Spa""",OH,3.0,3,Beauty & Spas;Nail Salons,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
8,0.014250,0.744750,0.241000,4.000000,"""Bath & Body Works""",NC,4.0,4,Shopping;Cosmetics & Beauty Supply;Beauty & Spas,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
12,0.040431,0.743097,0.216458,3.486111,"""Happy Moose Bar and Grill""",OH,3.5,72,Restaurants;Sports Bars;American (Traditional);Nightlife;Bars,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
13,0.045917,0.752750,0.201333,2.916667,"""Hungry Howie's Pizza""",NC,3.0,12,Restaurants;Sandwiches;Pizza,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152014,0.069500,0.786417,0.144167,3.416667,"""Dairy King""",OH,3.5,12,Food;Ice Cream & Frozen Yogurt,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152015,0.053625,0.835250,0.111125,3.000000,"""Mike The Plumber""",NC,3.0,8,Plumbing;Home Services;Water Heater Installation/Repair,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152022,0.168000,0.796000,0.036000,1.666667,"""Wendy's""",OH,1.5,3,Burgers;Restaurants;Fast Food,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152030,0.011000,0.679500,0.309500,5.000000,"""Serenity Hair Spa by Lauren""",NC,5.0,4,Hair Stylists;Hair Salons;Hair Extensions;Beauty & Spas;Permanent Makeup;Men's Hair Salons;Blow Dry/Out Services,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [29]:
df_filtered = df_filtered.replace(to_replace='Na', value=2)
df_filtered = df_filtered.replace(to_replace='True', value=1)
df_filtered = df_filtered.replace(to_replace='False', value=0)

In [30]:
cols = df_filtered.columns.tolist()
new_cols = cols[4:] + cols[0:4]
df_filtered = df_filtered[new_cols]

In [31]:
df_filtered['categories'] = df_filtered['categories'].astype(str)

In [32]:
df_clean = df_filtered.copy()
df_clean['categories'] = df_clean['categories'].apply(clean_text)
df_clean.to_csv('data/cleaned.csv')

In [33]:
df_clean

,name,state,stars_y,review_count,categories,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,...,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,neg,neu,pos,stars_x
1,"""Primal Brewery""",NC,4.0,47,food brewery,2,2,2,2,2,...,2,2,2,2,2,2,0.039043,0.718319,0.242809,3.851064
7,"""MV Nail Spa""",OH,3.0,3,beauty spa nail salon,2,2,0,2,2,...,2,2,2,2,2,2,0.081000,0.788000,0.131000,3.000000
8,"""Bath & Body Works""",NC,4.0,4,shopping cosmetic beauty supply beauty spa,2,2,0,2,2,...,2,2,2,2,2,2,0.014250,0.744750,0.241000,4.000000
12,"""Happy Moose Bar and Grill""",OH,3.5,72,restaurant sport bar american traditional nightlife bar,2,2,2,2,2,...,2,2,2,2,2,2,0.040431,0.743097,0.216458,3.486111
13,"""Hungry Howie's Pizza""",NC,3.0,12,restaurant sandwich pizza,2,2,2,2,0,...,2,2,2,2,2,2,0.045917,0.752750,0.201333,2.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152014,"""Dairy King""",OH,3.5,12,food ice cream frozen yogurt,2,2,2,2,2,...,2,2,2,2,2,2,0.069500,0.786417,0.144167,3.416667
152015,"""Mike The Plumber""",NC,3.0,8,plumbing home service water heater installation repair,2,2,2,1,2,...,2,2,2,2,2,2,0.053625,0.835250,0.111125,3.000000
152022,"""Wendy's""",OH,1.5,3,burger restaurant fast food,2,2,2,2,2,...,2,2,2,2,2,2,0.168000,0.796000,0.036000,1.666667
152030,"""Serenity Hair Spa by Lauren""",NC,5.0,4,hair stylist hair salon hair extension beauty spa permanent makeup men's hair salon blow dry service,2,2,2,2,2,...,2,2,2,2,2,2,0.011000,0.679500,0.309500,5.000000


In [34]:
one_hot = df_clean['categories'].str.get_dummies(sep=' ')

df_clean = pd.concat([df_clean, one_hot], axis=1)

df_clean.drop('categories', axis=1, inplace=True)

df_clean

,name,state,stars_y,review_count,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,...,wine,winery,wing,women's,wrap,yelp,yoga,yogurt,ziplining,zoo
1,"""Primal Brewery""",NC,4.0,47,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
7,"""MV Nail Spa""",OH,3.0,3,2,2,0,2,2,2,...,0,0,0,0,0,0,0,0,0,0
8,"""Bath & Body Works""",NC,4.0,4,2,2,0,2,2,2,...,0,0,0,0,0,0,0,0,0,0
12,"""Happy Moose Bar and Grill""",OH,3.5,72,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
13,"""Hungry Howie's Pizza""",NC,3.0,12,2,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152014,"""Dairy King""",OH,3.5,12,2,2,2,2,2,2,...,0,0,0,0,0,0,0,1,0,0
152015,"""Mike The Plumber""",NC,3.0,8,2,2,2,1,2,2,...,0,0,0,0,0,0,0,0,0,0
152022,"""Wendy's""",OH,1.5,3,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
152030,"""Serenity Hair Spa by Lauren""",NC,5.0,4,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [35]:
numeric_cols = df_clean.select_dtypes(include=['int', 'float']).columns
name_col= df_clean[['name']]
state_col= df_clean[['state']]
df_numeric = pd.concat([name_col, state_col, df_clean[numeric_cols]], axis=1)
df_clean = df_numeric
df_clean['name'] = df_clean['name'].str.replace(r'"', '')
df_clean

,name,state,stars_y,review_count,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,...,wine,winery,wing,women's,wrap,yelp,yoga,yogurt,ziplining,zoo
1,Primal Brewery,NC,4.0,47,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
7,MV Nail Spa,OH,3.0,3,2,2,0,2,2,2,...,0,0,0,0,0,0,0,0,0,0
8,Bath & Body Works,NC,4.0,4,2,2,0,2,2,2,...,0,0,0,0,0,0,0,0,0,0
12,Happy Moose Bar and Grill,OH,3.5,72,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
13,Hungry Howie's Pizza,NC,3.0,12,2,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152014,Dairy King,OH,3.5,12,2,2,2,2,2,2,...,0,0,0,0,0,0,0,1,0,0
152015,Mike The Plumber,NC,3.0,8,2,2,2,1,2,2,...,0,0,0,0,0,0,0,0,0,0
152022,Wendy's,OH,1.5,3,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
152030,Serenity Hair Spa by Lauren,NC,5.0,4,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df_clean = df_clean.filter(regex='^(?!Unnamed)')

In [37]:
df_clean.to_csv('final.csv', index=False)

In [38]:
#df_clean.to_csv('test.csv', index=False)

In [65]:
df_clean = pd.read_csv('test.csv')


In [66]:
df_clean = df_clean.drop('stars_y', axis=1)

In [67]:
nc = df_clean[df_clean['state'] == 'NC']
oh = df_clean[df_clean['state'] != 'NC']

In [41]:
#nc = nc.reset_index(drop=True)
#OH = OH.reset_index(drop=True)
#df_clean = df_clean.set_index('name', inplace=True)

In [68]:
oh = oh.set_index('name')

In [69]:
nc = nc.set_index('name')
nc

,state,review_count,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,...,wine,winery,wing,women's,wrap,yelp,yoga,yogurt,ziplining,zoo
name,,,,,,,,,,,,,,,,,,,,,
Primal Brewery,NC,47,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
Bath & Body Works,NC,4,2,2,0,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
Hungry Howie's Pizza,NC,12,2,2,2,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Pronto Pizza,NC,4,2,2,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Charlotte Root Canal Center,NC,7,2,2,2,1,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sky Nail Bar & Spa,NC,22,2,2,0,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
Bad Daddys,NC,12,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
Mike The Plumber,NC,8,2,2,2,1,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [70]:
nc = nc.drop('state', axis=1)
oh = oh.drop('state', axis=1)


In [71]:
nc = nc.drop('review_count', axis=1)
oh = oh.drop('review_count', axis=1)

In [79]:
def find_similar_rows(nc, oh, row_index):
    # Get the specified row from the first dataframe
    row = nc.iloc[row_index]

    # Compute the Euclidean distance between the specified row and all rows in the second dataframe
    distances = np.linalg.norm(oh - row, axis=1)

    # Find the indices of the 5 smallest distances
    min_distance_indices = np.argsort(distances)[:5]

    # Return the top 5 rows in the second dataframe with the smallest distance to the specified row
    return oh.iloc[min_distance_indices]

In [54]:
#nc.to_csv('Ohio.csv', index=False)
#oh.to_csv('NorthCarolina.csv', index=False)

In [80]:
row_index = nc.index.get_loc('Dolce Vita Wine Bar')
row_index

8279

In [81]:
nc.iloc[8279]

AcceptsInsurance              2.0
ByAppointmentOnly             2.0
BusinessAcceptsCreditCards    2.0
BusinessParking_garage        2.0
BusinessParking_street        2.0
                             ... 
yelp                          0.0
yoga                          0.0
yogurt                        0.0
ziplining                     0.0
zoo                           0.0
Name: Dolce Vita Wine Bar, Length: 1105, dtype: float64

In [85]:
def find_similar_rows(nc, oh, row_index):
    # Get the specified row from the first dataframe
    row = nc.iloc[row_index]

    # Compute the Euclidean distance between the specified row and all rows in the second dataframe
    distances = np.linalg.norm(oh - row, axis=1)

    # Find the indices of the 5 smallest distances
    min_distance_indices = np.argsort(distances)[:5]

    # Return the index values of the top 5 rows in the second dataframe with the smallest distance to the specified row
    return list(oh.index[min_distance_indices])

In [86]:
similar_row = find_similar_rows(nc, oh, 8279)
similar_row

['Felice', 'Cantine', 'Camino', 'Wine Brew and More', 'The Bier Haus']

In [91]:
st.title('Nom-Native')

DeltaGenerator()